# BMW Service Manual Model - Inference Testing

This notebook provides systematic testing of the finetuned Llama-3.2-3B model.

## What This Notebook Does
1. Loads your finetuned model from HuggingFace Hub
2. Tests predefined queries across all task types
3. Evaluates on validation set with ground truth comparison
4. Provides error analysis and accuracy metrics

## Requirements
- GPU: Optional (CPU inference works, just slower)
- Model pushed to HuggingFace Hub
- Validation data uploaded to Google Drive (optional)

## Cell 1: Setup

In [ ]:
# Install required packages
!pip install -q transformers peft accelerate bitsandbytes

# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
import json
from typing import List, Dict, Any
import time

print("✅ Packages installed and imported")

## Cell 2: Load Model from HuggingFace Hub

**⚠️ Important**: Change `model_id` to your HuggingFace model ID!

In [ ]:
# CHANGE THIS to your model ID
model_id = "your-username/bmw-e30-m3-service-manual"

print(f"🔄 Loading model: {model_id}\n")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
print("✅ Tokenizer loaded")

# Load base model
print("🔄 Loading base model (Llama-3.2-3B-Instruct)...")
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16
)
print("✅ Base model loaded")

# Load LoRA adapter
print(f"🔄 Loading LoRA adapter from {model_id}...")
model = PeftModel.from_pretrained(base_model, model_id)
print("✅ LoRA adapter loaded")

# Set to eval mode
model.eval()

print("\n✅ Model loaded successfully and ready for inference!")
print(f"Device: {model.device}")

## Cell 3: Inference Function

In [ ]:
def generate_response(
    prompt: str,
    max_new_tokens: int = 150,
    temperature: float = 0.7,
    top_p: float = 0.9,
    verbose: bool = False
) -> str:
    """
    Generate response for a given prompt.
    
    Args:
        prompt: Input prompt with task prefix (e.g., "[SPEC] What is...")
        max_new_tokens: Maximum tokens to generate
        temperature: Sampling temperature (0.1=factual, 0.7=balanced, 1.0=creative)
        top_p: Nucleus sampling parameter
        verbose: Print generation details
    
    Returns:
        Generated response text
    """
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    if verbose:
        print(f"Full prompt:\n{text}\n")
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True if temperature > 0 else False,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    elapsed = time.time() - start_time
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the assistant's response
    if "assistant" in response:
        response = response.split("assistant")[-1].strip()
    
    if verbose:
        print(f"Generation time: {elapsed:.2f}s")
        print(f"Tokens generated: {len(outputs[0]) - len(inputs['input_ids'][0])}")
    
    return response

print("✅ Inference function defined")

# Quick test
print("\n🧪 Quick test:")
test_response = generate_response("[SPEC] What is the torque?", max_new_tokens=50, verbose=False)
print(f"Response: {test_response}")

## Cell 4: Load Validation Set (Optional)

Upload `data/val.jsonl` to Google Drive at `/MyDrive/llm3/data/val.jsonl`

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load validation data
val_path = '/content/drive/MyDrive/llm3/data/val.jsonl'

try:
    val_data = []
    with open(val_path) as f:
        for line in f:
            val_data.append(json.loads(line))
    
    print(f"✅ Loaded {len(val_data)} validation examples")
    
    # Show distribution
    from collections import Counter
    task_counts = Counter(ex['meta']['task'] for ex in val_data)
    print("\n📊 Validation set distribution:")
    for task, count in sorted(task_counts.items()):
        print(f"  {task:<20} {count:>4}")
    
except FileNotFoundError:
    print("⚠️ Validation data not found. Upload val.jsonl to Google Drive.")
    print("   You can still run manual tests in the next cells.")
    val_data = []

## Cell 5: Predefined Test Cases by Task Type

In [ ]:
# Define test cases from each task type
test_cases = [
    {
        "task": "SPEC",
        "query": "What is the torque for cylinder head bolts?",
        "expected": "Should return value with unit (e.g., '45 Nm')"
    },
    {
        "task": "SPEC",
        "query": "What is the engine displacement?",
        "expected": "Should return value with unit (e.g., '2.3 L')"
    },
    {
        "task": "PROCEDURE",
        "query": "How do you adjust valve clearance?",
        "expected": "Should return numbered steps (1. 2. 3...)"
    },
    {
        "task": "PROCEDURE",
        "query": "How do you bleed the brake system?",
        "expected": "Should return numbered steps"
    },
    {
        "task": "EXPLANATION",
        "query": "Explain the Motronic control unit operation",
        "expected": "Should give technical explanation with details"
    },
    {
        "task": "EXPLANATION",
        "query": "Explain the fuel injection system",
        "expected": "Should describe system operation"
    },
    {
        "task": "WIRING",
        "query": "What are the wiring details for terminal 15u routing?",
        "expected": "Should describe wire routing/connections"
    },
    {
        "task": "SPEC",
        "query": "What is the oil capacity?",
        "expected": "Should return value with unit (e.g., '5.0 L')"
    },
]

# Run tests
print("🧪 Running predefined test cases:\n")
print("=" * 100)

for i, test in enumerate(test_cases, 1):
    prompt = f"[{test['task']}] {test['query']}"
    
    print(f"\n{i}. Task: {test['task']}")
    print(f"   Query: {test['query']}")
    print(f"   Expected: {test['expected']}")
    
    response = generate_response(prompt, max_new_tokens=150, temperature=0.7)
    
    print(f"   Response: {response}")
    print("-" * 100)

print("\n✅ Test cases complete")

## Cell 6: Validation Set Evaluation (Sample)

In [ ]:
# Test on a sample of validation data
import random

if len(val_data) == 0:
    print("⚠️ No validation data loaded. Upload val.jsonl to run this test.")
else:
    sample_size = 10
    sample_indices = random.sample(range(len(val_data)), min(sample_size, len(val_data)))
    
    print(f"📊 Evaluating on {sample_size} random validation examples:\n")
    print("=" * 100)
    
    for idx in sample_indices:
        example = val_data[idx]
        task = example['meta']['task']
        instruction = example['instruction']
        ground_truth = example['output']
        
        prompt = f"[{task.upper()}] {instruction}"
        response = generate_response(prompt, max_new_tokens=150, temperature=0.7)
        
        # Simple matching
        exact_match = response.strip().lower() == ground_truth.strip().lower()
        partial_match = ground_truth.strip().lower() in response.lower()
        
        print(f"\nTask: {task}")
        print(f"Query: {instruction}")
        print(f"Ground Truth: {ground_truth}")
        print(f"Model Output: {response}")
        print(f"Exact Match: {'✅' if exact_match else '❌'}")
        print(f"Partial Match: {'✅' if partial_match else '❌'}")
        print("-" * 100)
    
    print("\n✅ Sample evaluation complete")

## Cell 7: Full Validation Set Evaluation with Metrics

In [ ]:
if len(val_data) == 0:
    print("⚠️ No validation data loaded. Upload val.jsonl to run this test.")
else:
    # Evaluate on subset (or full set if you have time)
    eval_size = min(50, len(val_data))  # Evaluate on first 50 examples
    
    print(f"📈 Running full validation evaluation on {eval_size} examples...\n")
    
    results = {
        "exact_match": 0,
        "partial_match": 0,
        "no_match": 0,
        "by_task": {}
    }
    
    errors = []
    
    for i, example in enumerate(val_data[:eval_size]):
        task = example['meta']['task']
        instruction = example['instruction']
        ground_truth = example['output']
        
        prompt = f"[{task.upper()}] {instruction}"
        response = generate_response(prompt, max_new_tokens=150, temperature=0.7)
        
        # Matching logic
        exact = response.strip().lower() == ground_truth.strip().lower()
        partial = ground_truth.strip().lower() in response.lower()
        
        if exact:
            results["exact_match"] += 1
            match_type = "exact"
        elif partial:
            results["partial_match"] += 1
            match_type = "partial"
        else:
            results["no_match"] += 1
            match_type = "no_match"
            errors.append({
                "task": task,
                "instruction": instruction,
                "ground_truth": ground_truth,
                "response": response
            })
        
        # Track by task
        if task not in results["by_task"]:
            results["by_task"][task] = {"exact": 0, "partial": 0, "no_match": 0, "total": 0}
        results["by_task"][task][match_type] += 1
        results["by_task"][task]["total"] += 1
        
        # Progress
        if (i + 1) % 10 == 0:
            print(f"Evaluated {i + 1}/{eval_size}...")
    
    # Print results
    total = results["exact_match"] + results["partial_match"] + results["no_match"]
    
    print("\n" + "=" * 100)
    print("📊 OVERALL RESULTS")
    print("=" * 100)
    print(f"Total evaluated: {total}")
    print(f"Exact matches:   {results['exact_match']} ({results['exact_match']/total*100:.1f}%)")
    print(f"Partial matches: {results['partial_match']} ({results['partial_match']/total*100:.1f}%)")
    print(f"No matches:      {results['no_match']} ({results['no_match']/total*100:.1f}%)")
    print(f"\nAccuracy (exact + partial): {(results['exact_match'] + results['partial_match'])/total*100:.1f}%")
    
    print("\n" + "=" * 100)
    print("📊 RESULTS BY TASK TYPE")
    print("=" * 100)
    
    for task, stats in sorted(results["by_task"].items()):
        total_task = stats["total"]
        print(f"\n{task.upper()}:")
        print(f"  Total:   {total_task}")
        print(f"  Exact:   {stats['exact']} ({stats['exact']/total_task*100:.1f}%)")
        print(f"  Partial: {stats['partial']} ({stats['partial']/total_task*100:.1f}%)")
        print(f"  Wrong:   {stats['no_match']} ({stats['no_match']/total_task*100:.1f}%)")
        print(f"  Accuracy: {(stats['exact'] + stats['partial'])/total_task*100:.1f}%")
    
    # Show error examples
    if errors:
        print("\n" + "=" * 100)
        print(f"❌ ERROR EXAMPLES (showing first 5 of {len(errors)})")
        print("=" * 100)
        
        for i, err in enumerate(errors[:5], 1):
            print(f"\n{i}. Task: {err['task']}")
            print(f"   Query: {err['instruction']}")
            print(f"   Expected: {err['ground_truth']}")
            print(f"   Got: {err['response']}")
            print("-" * 100)
    
    print("\n✅ Evaluation complete")

## Cell 8: Interactive Testing

Test your own queries interactively!

In [ ]:
# Interactive testing
print("🎮 Interactive Testing Mode")
print("Enter your queries below. Format: [TASK] question")
print("Tasks: SPEC, PROCEDURE, EXPLANATION, WIRING, TROUBLESHOOTING")
print("Type 'quit' to exit\n")

while True:
    try:
        query = input("\nYour query: ")
        
        if query.lower() in ['quit', 'exit', 'q']:
            print("👋 Goodbye!")
            break
        
        if not query.strip():
            continue
        
        # Check if query has task prefix
        if not query.startswith('['):
            print("⚠️ Query should start with task prefix like [SPEC] or [PROCEDURE]")
            continue
        
        response = generate_response(query, max_new_tokens=150, temperature=0.7)
        print(f"\n💬 Response: {response}")
        
    except KeyboardInterrupt:
        print("\n👋 Goodbye!")
        break
    except Exception as e:
        print(f"❌ Error: {e}")

## Cell 9: Compare Temperatures

Test how different temperature settings affect output quality

In [ ]:
# Compare different temperatures
test_query = "[SPEC] What is the torque for cylinder head bolts?"
temperatures = [0.1, 0.5, 0.7, 1.0]

print(f"🌡️ Testing different temperatures on query: {test_query}\n")
print("=" * 100)

for temp in temperatures:
    print(f"\nTemperature: {temp}")
    response = generate_response(test_query, max_new_tokens=100, temperature=temp)
    print(f"Response: {response}")
    print("-" * 100)

print("\n💡 Lower temperature (0.1-0.3): More deterministic, factual")
print("💡 Medium temperature (0.5-0.7): Balanced, good for most cases")
print("💡 High temperature (0.8-1.0): More creative, varied")

## Summary

This notebook provided systematic testing of your finetuned BMW service manual model.

### Key Metrics to Track
1. **Exact match rate**: How often model outputs match ground truth exactly
2. **Partial match rate**: How often model outputs contain correct information
3. **Per-task accuracy**: Performance breakdown by task type
4. **Error patterns**: Common failure modes

### Expected Performance
- **SPEC tasks**: Should have highest exact match rate (90%+)
- **PROCEDURE tasks**: Should produce numbered steps (80%+)
- **EXPLANATION tasks**: Should be coherent and factually accurate (70%+)
- **WIRING/TROUBLESHOOTING**: Should provide relevant technical details

### Next Steps
1. If accuracy < 70%: Consider training longer or with larger LoRA rank
2. If specific tasks underperform: Check if training data had enough examples
3. If outputs are too random: Lower temperature (0.3-0.5)
4. If outputs are too repetitive: Increase temperature slightly (0.7-0.8)

### Deployment
Once satisfied with performance:
1. Create inference API (HuggingFace Inference Endpoints)
2. Build demo interface (Gradio/Streamlit)
3. Share with BMW enthusiasts/mechanics for feedback
4. Iterate based on real-world usage